In [435]:
%load_ext autoreload
%autoreload 2

from kernel_lib import *
from matrix import Matrix
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [436]:
vocab = ["This", "is", "a", "sentence"]
vocab_size = len(vocab)

pos_enc_sequence_len = 10
token_dims = vocab_size

In [437]:
pos_encodings = Matrix(pos_enc_sequence_len, token_dims, np.float32, gpu=True)
pos_encodings.alloc_on_gpu()
gen_pos_encodings(pos_encodings.a_gpu,
                  np.int32(pos_encodings.num_rows),
                  np.int32(pos_encodings.num_cols),
                  block=(pos_encodings.num_cols, pos_encodings.num_rows, 1))
cuda.Context.synchronize()

print(pos_encodings)

[[ 0.          1.          0.          1.        ]
 [ 0.841471    0.5403023   0.00999983  0.99995   ]
 [ 0.90929747 -0.4161468   0.01999867  0.9998    ]
 [ 0.14112    -0.9899925   0.0299955   0.99955004]
 [-0.7568025  -0.65364367  0.03998933  0.9992001 ]
 [-0.9589243   0.28366217  0.04997917  0.99875027]
 [-0.2794155   0.96017027  0.059964    0.99820054]
 [ 0.6569866   0.75390226  0.06994285  0.997551  ]
 [ 0.98935825 -0.14550003  0.07991469  0.99680173]
 [ 0.4121185  -0.91113025  0.08987855  0.9959527 ]]


In [438]:
rows_np = np.arange(pos_enc_sequence_len)
rows_np = rows_np[:, np.newaxis]

pos_cols_np = np.arange(token_dims)
pos_cols_np = pos_cols_np[np.newaxis, :]
pos_cols_np = np.power(10000, (2 * (pos_cols_np // 2)) / token_dims)

pos_enc_pre_sin = rows_np / pos_cols_np

pos_encoding_np = np.zeros(pos_enc_pre_sin.shape)
pos_encoding_np[:, 0::2] = np.sin(pos_enc_pre_sin[:, 0::2])
pos_encoding_np[:, 1::2] = np.cos(pos_enc_pre_sin[:, 1::2])

if (pos_encodings.compare(pos_encoding_np)):
  print("They are similar!")
else:
  print("They are not similar!")

They are similar!


In [439]:
sentence = "This is a sentence"
sentence_toks = [0, 1, 2, 3] # Straight forward
word2tok = {"This" : 0, "is" : 1, "a" : 2, "sentence" : 3}

In [440]:
def xavier_uniform(fan_in, fan_out):
  return math.sqrt(6 / (fan_in + fan_out))

In [441]:
embeddings = Matrix(vocab_size, token_dims, np.float32, gpu=True)
embeddings.alloc_on_gpu()
embeddings_scale = xavier_uniform(embeddings.num_rows, embeddings.num_cols)
embeddings.init_uniform_rand(embeddings_scale)

cuda.Context.synchronize()

print(f"{embeddings=}")


embeddings=[[ 0.41607213  0.72918266 -0.7984399   0.81226754]
 [ 0.736365   -0.09292436  0.28980532 -0.67561984]
 [-0.0515829   0.0228521   0.47834936 -0.35547632]
 [ 0.37067556 -0.24508198  0.31422633 -0.3602407 ]]


In [442]:
embeddings.copy_d_to_h()
embeddings_np = embeddings.a_host.copy()
print(embeddings_np)

[[ 0.41607213  0.72918266 -0.7984399   0.81226754]
 [ 0.736365   -0.09292436  0.28980532 -0.67561984]
 [-0.0515829   0.0228521   0.47834936 -0.35547632]
 [ 0.37067556 -0.24508198  0.31422633 -0.3602407 ]]


In [443]:
#TODO: Add a function that adds two matrices but has the ability to "scale" down matrices based on which one is bigger, etc
# Special "trimmed" matrix add...
kernel_code = """
// Assumes embedding matrix has been sized such that Dim(embedding_matrix) < Dim(pos_enc)
extern "C" __global__ void add_pos_enc_and_embed(float* embedding_matrix, float* pos_enc, float* output, int N) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx < N) {
    output[idx] = embedding_matrix[idx] + pos_enc[idx];
  }
}
"""

mod = SourceModule(kernel_code,
                   no_extern_c=True,
                   options=["-std=c++11",
                           "-Xcompiler",
                           "-fPIC"])

add_pos_enc_and_embed = mod.get_function("add_pos_enc_and_embed")

In [444]:
embeddings_w_pos = Matrix(embeddings.num_rows, embeddings.num_cols, np.float32, gpu=True)
embeddings_w_pos.alloc_on_gpu()
add_pos_enc_and_embed(embeddings.a_gpu,
                      pos_encodings.a_gpu,
                      embeddings_w_pos.a_gpu,
                      np.int32(embeddings.num_elements()),
                      block=(embeddings.num_elements(), 1, 1))
cuda.Context.synchronize()

print(f"{embeddings_w_pos=}")

embeddings_w_pos=[[ 0.41607213  1.7291827  -0.7984399   1.8122675 ]
 [ 1.577836    0.44737792  0.29980516  0.32433015]
 [ 0.8577146  -0.39329472  0.49834803  0.6443237 ]
 [ 0.5117956  -1.2350745   0.34422183  0.63930935]]


In [445]:
trimmed_pos_enc_np = pos_encoding_np[:4, :]
embeddings_w_pos_np = embeddings_np + trimmed_pos_enc_np

if (embeddings_w_pos.compare(embeddings_w_pos_np)):
  print("Embeddings are similar!")
else:
  print(f"{embeddings_np=}")
  print(f"{trimmed_pos_enc_np=}")
  print(f"{embeddings_w_pos_np=}")
  print("Embeddings not similar!")

Embeddings are similar!


In [446]:
test_a = Matrix(4,4,np.float32,gpu=True)
test_a.alloc_on_gpu()
test_a.init_incremental()

test_b = Matrix(4,4,np.float32,gpu=True)
test_b.alloc_on_gpu()
test_b.init_incremental()

test_c = test_a * test_b
print(test_c)

[[ 56.  62.  68.  74.]
 [152. 174. 196. 218.]
 [248. 286. 324. 362.]
 [344. 398. 452. 506.]]


In [447]:
print(f"Initially: {test_c=}")
test_c = test_c / 2
print(f"After scalar divide: {test_c=}")

Returning the cached matrix value!
Initially: test_c=[[ 56.  62.  68.  74.]
 [152. 174. 196. 218.]
 [248. 286. 324. 362.]
 [344. 398. 452. 506.]]
After scalar divide: test_c=[[ 28.  31.  34.  37.]
 [ 76.  87.  98. 109.]
 [124. 143. 162. 181.]
 [172. 199. 226. 253.]]


In [448]:
# Embedding matrix = [vocab_size, token_dims]
# Technically you can make swap the dimensions of this and it will still work
# One way requires a transpose, the other doesn't
# Weights: [3 * token_dims, token_dims]

#TODO: Make a weight transpose instead for learnings... Even though a bit less efficient...
# weights = Matrix(3 * embeddings.num_cols, embeddings.num_cols, np.float32, gpu=True)
# weights.alloc_on_gpu()
weights_t = Matrix(embeddings_w_pos.num_cols, 3 * embeddings_w_pos.num_cols, np.float32, gpu=True)
weights_t.alloc_on_gpu()
weights_scale = xavier_uniform(weights_t.num_rows, weights_t.num_cols)
weights_t.init_uniform_rand(weights_scale)

# QKV matrix = [vocab_size, 3 * token_dims]
QKV = embeddings_w_pos * weights_t

bias_scale = xavier_uniform(QKV.num_cols, 1)

b = Matrix(QKV.num_cols, 1, np.float32, gpu=True)
b.alloc_on_gpu()
b.init_uniform_rand(bias_scale)

QKV_b = Matrix(QKV.num_rows, QKV.num_cols, np.float32, gpu=True)
QKV_b.alloc_on_gpu()

add_matrix_w_vector(QKV.a_gpu,
                    b.a_gpu,
                    np.int32(QKV.num_rows),
                    np.int32(QKV.num_cols),
                    QKV_b.a_gpu,
                    block=(QKV.num_cols, QKV.num_rows, 1))

In [449]:
b.copy_d_to_h()
b_np = b.a_host.copy()
b_np = b_np.T

# weights_t_np = np.random.uniform(low=-weights_scale, high=weights_scale, size=(weights_t.num_rows, weights_t.num_cols))
weights_t.copy_d_to_h()
weights_t_np = weights_t.a_host.copy()

QKV_np = embeddings_w_pos_np @ weights_t_np
QKV_b_np = QKV_np + b_np

if (not QKV_b.compare(QKV_b_np)):
  print("QKV_b are similar!")
else:
  print(f"{QKV_b=}")
  print(f"{QKV_b_np=}")
  print("QKV_b not similar!")

QKV_b=[[ 0.5858841  -0.07326937 -1.5218644   0.3128903  -0.14234573  1.1255311
  -0.9151788  -1.2557507  -1.4943337   0.22304492  0.5137239  -0.76285994]
 [ 0.7072086   1.4134607  -1.5254618   1.6387186   1.2436802   0.12450986
   0.19428861 -1.266494   -0.5250175   0.16229238  0.92345756 -0.8253772 ]
 [ 0.10867439  1.2466791  -1.4293613   1.5948985   0.989475   -0.02572972
   0.2553985  -0.845855   -0.43513     0.07947831  0.9863903  -0.64482915]
 [-0.16197938  1.1431178  -1.4462063   1.5293294   0.98203295 -0.33542147
   0.57010895 -0.66561127 -0.07674776 -0.0632465   1.0501893  -0.44635892]]
QKV_b_np=array([[ 0.58588406, -0.07326933, -1.52186439,  0.3128903 , -0.14234572,
         1.12553115, -0.91517874, -1.2557506 , -1.49433365,  0.22304491,
         0.51372392, -0.7628599 ],
       [ 0.70720858,  1.4134607 , -1.52546181,  1.63871851,  1.24368024,
         0.12450988,  0.19428861, -1.26649397, -0.52501754,  0.16229238,
         0.9234576 , -0.82537725],
       [ 0.10867436,  1.246

In [450]:
test_big_matrix = Matrix(4, 12, np.float32, gpu=True)
test_big_matrix.alloc_on_gpu()
test_big_matrix.init_incremental()

small_1 = Matrix(4, 4, np.float32, gpu=True)
small_1.set_gpu_matrix(test_big_matrix.a_gpu, stride=test_big_matrix.num_cols, start_idx=(0 * test_big_matrix.num_cols) / 3)

small_2 = Matrix(4, 4, np.float32, gpu=True)
small_2.set_gpu_matrix(test_big_matrix.a_gpu, stride=test_big_matrix.num_cols, start_idx=(0 * test_big_matrix.num_cols) / 3)

small_3 = Matrix(4, 4, np.float32, gpu=True)
small_3.set_gpu_matrix(test_big_matrix.a_gpu, stride=test_big_matrix.num_cols, start_idx=(0 * test_big_matrix.num_cols) / 3)

print(f"{small_1=}")
print(f"{small_2=}")
print(f"{small_3=}")

small_1=[[ 0.  1.  2.  3.]
 [12. 13. 14. 15.]
 [24. 25. 26. 27.]
 [36. 37. 38. 39.]]
small_2=[[ 0.  1.  2.  3.]
 [12. 13. 14. 15.]
 [24. 25. 26. 27.]
 [36. 37. 38. 39.]]
small_3=[[ 0.  1.  2.  3.]
 [12. 13. 14. 15.]
 [24. 25. 26. 27.]
 [36. 37. 38. 39.]]


In [451]:
Q = Matrix(QKV_b.num_rows, QKV_b.num_cols / 3, np.float32, gpu=True)
Q.set_gpu_matrix(QKV_b.a_gpu, stride=QKV_b.num_cols, start_idx=(0 * QKV_b.num_cols) / 3)

K = Matrix(QKV_b.num_rows, QKV_b.num_cols / 3, np.float32, gpu=True)
K.set_gpu_matrix(QKV_b.a_gpu, stride=QKV_b.num_cols, start_idx=(1 * QKV_b.num_cols) / 3)

V = Matrix(QKV_b.num_rows, QKV_b.num_cols / 3, np.float32, gpu=True)
V.set_gpu_matrix(QKV_b.a_gpu, stride=QKV_b.num_cols, start_idx=(2 * QKV_b.num_cols) / 3)

score_scaled = (Q * K.transpose()) / math.sqrt(Q.num_cols)

In [452]:
split_dim = int(QKV_b_np.shape[1] / 3)

Q_np = QKV_b_np[:, : split_dim]
K_np = QKV_b_np[:, split_dim : 2 * split_dim]
V_np = QKV_b_np[:, 2 * split_dim :]

if (Q.compare(Q_np)):
  print("Q is similar!")
else:
  print(f"{Q=}")
  print(f"{Q_np=}")
  print("Q is not similar")

if (K.compare(K_np)):
  print("K is similar!")
else:
  print(f"{K=}")
  print(f"{K_np=}")
  print("K is not similar")

if (V.compare(V_np)):
  print("V is similar!")
else:
  print(f"{V=}")
  print(f"{V_np=}")
  print("V is not similar")

Q=[[ 0.5858841  -0.07326937 -1.5218644   0.3128903 ]
 [ 0.7072086   1.4134607  -1.5254618   1.6387186 ]
 [ 0.10867439  1.2466791  -1.4293613   1.5948985 ]
 [-0.16197938  1.1431178  -1.4462063   1.5293294 ]]
Q_np=array([[ 0.58588406, -0.07326933, -1.52186439,  0.3128903 ],
       [ 0.70720858,  1.4134607 , -1.52546181,  1.63871851],
       [ 0.10867436,  1.24667903, -1.4293614 ,  1.59489841],
       [-0.16197938,  1.14311773, -1.44620626,  1.52932941]])
Q is not similar
K=[[-0.14234573  1.1255311  -0.9151788  -1.2557507 ]
 [ 1.2436802   0.12450986  0.19428861 -1.266494  ]
 [ 0.989475   -0.02572972  0.2553985  -0.845855  ]
 [ 0.98203295 -0.33542147  0.57010895 -0.66561127]]
K_np=array([[-0.14234572,  1.12553115, -0.91517874, -1.2557506 ],
       [ 1.24368024,  0.12450988,  0.19428861, -1.26649397],
       [ 0.98947497, -0.02572972,  0.25539853, -0.84585492],
       [ 0.98203293, -0.33542146,  0.57010891, -0.66561129]])
K is not similar
V=[[-1.4943337   0.22304492  0.5137239  -0.76285994]

In [453]:
score_scaled = (Q * K.transpose()) / math.sqrt(Q.num_cols)
score_scaled_np = (Q_np @ K_np.T) / math.sqrt(Q.num_cols)

print(f"{score_scaled=}")
print(f"{score_scaled_np=}")

score_scaled=[[ 0.41700038  0.01378758 -0.03586942 -0.2379791 ]
 [ 0.41423714 -0.65813804 -0.55616087 -0.870016  ]
 [ 0.34651658 -1.003629   -0.81932783 -1.0939575 ]
 [ 0.3563763  -1.1384946  -0.9263183  -1.1924647 ]]
score_scaled_np=array([[ 0.41700038,  0.01378754, -0.03586946, -0.23797911],
       [ 0.4142372 , -0.6581379 , -0.55616079, -0.87001593],
       [ 0.34651665, -1.00362892, -0.81932781, -1.09395758],
       [ 0.35637629, -1.13849449, -0.9263182 , -1.19246465]])


In [454]:
test_a = Matrix(4,4,np.float32,gpu=True)
test_a.alloc_on_gpu()
test_a.init_incremental()

test_output = Matrix(4,1,np.float32,gpu=True)
test_output.alloc_on_gpu()
test_output.init_incremental()

print(f"{test_a=}")
print(f"Before: {test_output=}")

matrix_row_wise_add(test_a.a_gpu,
        np.int32(test_a.num_rows),
        np.int32(test_a.num_cols),
        test_output.a_gpu,
        block=(test_a.num_cols,test_a.num_rows,1),
        shared=test_a.num_elements() * test_a.dtype().nbytes)

print(f"After: {test_output=}")

test_a=[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
Before: test_output=[[0.]
 [1.]
 [2.]
 [3.]]
Returning the cached matrix value!
After: test_output=[[0.]
 [1.]
 [2.]
 [3.]]


In [455]:
score_scaled_row_sum = Matrix(score_scaled.num_rows, 1, np.float32, gpu=True)
score_scaled_row_sum.alloc_on_gpu()

matrix_row_wise_add(score_scaled.a_gpu,
        np.int32(score_scaled.num_rows),
        np.int32(score_scaled.num_cols),
        score_scaled_row_sum.a_gpu,
        block=(score_scaled.num_cols, score_scaled.num_rows, 1),
        shared=score_scaled.num_elements() * score_scaled.dtype().nbytes)

print(f"{score_scaled_row_sum=}")

score_scaled_row_sum=[[ 0.15693945]
 [-1.6700778 ]
 [-2.5703979 ]
 [-2.9009013 ]]


In [456]:
score_scaled_row_sum_np = np.sum(score_scaled_np, axis=1)
print(f"{score_scaled_row_sum_np=}")

score_scaled_row_sum_np=array([ 0.15693935, -1.67007742, -2.57039766, -2.90090105])


In [457]:
score_scaled_row_max = Matrix(score_scaled.num_rows, 1, np.float32, gpu=True)
score_scaled_row_max.alloc_on_gpu()

matrix_row_wise_max(score_scaled.a_gpu,
        np.int32(score_scaled.num_rows),
        np.int32(score_scaled.num_cols),
        score_scaled_row_max.a_gpu,
        block=(score_scaled.num_cols, score_scaled.num_rows, 1),
        shared=score_scaled.num_elements() * score_scaled.dtype().nbytes)

print(f"{score_scaled_row_max=}")

score_scaled_row_max=[[0.41700038]
 [0.41423714]
 [0.34651658]
 [0.3563763 ]]


In [458]:
score_scaled_row_max_np = np.max(score_scaled_np, axis=1)
print(f"{score_scaled_row_max_np=}")

score_scaled_row_max_np=array([0.41700038, 0.4142372 , 0.34651665, 0.35637629])


In [459]:
out = Matrix(score_scaled.num_rows, score_scaled.num_cols, np.float32, gpu=True)
out.alloc_on_gpu()
out.init_incremental()

matrix_bytes = score_scaled.num_elements() * score_scaled.dtype().nbytes
shared_mem_bytes = int((3 * matrix_bytes) / 2)

fused_softmax(score_scaled.a_gpu,
              np.int32(score_scaled.num_rows),
              np.int32(score_scaled.num_cols),
              out.a_gpu,
              block=(score_scaled.num_cols, score_scaled.num_rows, 1),
              shared=shared_mem_bytes)

print(f"{out=}")

out=[[0.35417998 0.23665237 0.22518794 0.18397976]
 [0.50050443 0.17127    0.18965724 0.1385683 ]
 [0.5531961  0.14338982 0.17240874 0.13100538]
 [0.5834099  0.13084576 0.16177322 0.12397116]]


In [460]:
# softmax = exp(score_scaled - score_scaled_row_max) / score_scaled_row_sum_np
score_shifted_np = np.exp(score_scaled_np - score_scaled_row_max_np)
score_shifted_sum_np = np.sum(score_shifted_np, axis=1, keepdims=True)
score_softmaxed_np = score_shifted_np / score_shifted_sum_np
print(f"{score_softmaxed_np=}")
# print(f"{score_scaled_softmaxed=}")

score_softmaxed_np=array([[0.34433264, 0.23070931, 0.2349146 , 0.19004345],
       [0.48925908, 0.16788517, 0.19893495, 0.1439208 ],
       [0.54172503, 0.14080503, 0.18116312, 0.13630682],
       [0.57201307, 0.12864469, 0.17019606, 0.12914618]])


In [461]:
# print(score_scaled_softmaxed.shape)
# print(V.shape)

attention_output = score_scaled_softmaxed * V
print(f"{attention_output=}")

NameError: name 'score_scaled_softmaxed' is not defined

In [ ]:
print(f"{embeddings_w_pos=}")
print(f"{attention_output=}")
add = embeddings_w_pos + attention_output
print(f"{add}")

embeddings_w_pos=[[ 0.41607213  1.7291827  -0.7984399   1.8122675 ]
 [ 1.577836    0.44737792  0.29980516  0.32433015]
 [ 0.8577146  -0.39329472  0.49834803  0.6443237 ]
 [ 0.5117956  -1.2350745   0.34422183  0.63930935]]
Returning the cached matrix value!
attention_output=[[-0.7346844   0.13233826 -0.07186561 -0.24650136]
 [-0.09149098  0.00931656  4.813669   -2.5249176 ]
 [-0.3978891   0.07214858  2.3947058  -1.3989027 ]
 [-0.41401434  0.07653806  2.2442029  -1.3293835 ]]
[[-0.31861228  1.861521   -0.87030554  1.5657662 ]
 [ 1.486345    0.45669448  5.1134744  -2.2005875 ]
 [ 0.4598255  -0.32114613  2.8930538  -0.75457895]
 [ 0.09778124 -1.1585364   2.5884247  -0.69007415]]
